In [5]:
import gym
import keras
import datetime as dt
import tensorflow as tf
import random
import numpy as np
import math
from tensorflow.keras.layers import Dense

STORE_PATH = '/run'
MAX_EPSILON = 1
MIN_EPSILON = 0.01
LAMBDA = 0.0005
GAMMA = 0.95
BATCH_SIZE = 32
TAU = 0.08
RANDOM_REWARD_STD = 1.0
env = gym.make("CartPole-v1")
# env = gym.make("MountainCar-v0")
state_size = 4
num_actions = env.action_space.n



class Memory():
    def __init__(self, max_memory):
        self._max_memory = max_memory
        self._samples = []
    def add_sample(self, sample):
        self._samples.append(sample)
        if len(self._samples) > self._max_memory:
            self._samples.pop(0)
    def sample(self, no_samples):
        if no_samples > len(self._samples):
            return random.sample(self._samples, len(self._samples))
        else:
            return random.sample(self._samples, no_samples)
    @property
    def num_samples(self):
        return len(self._samples)
memory = Memory(50000)


class QN(tf.keras.Model):
    def __init__(self):
        super(QN,self).__init__()
        self.l1 = Dense(30, input_shape=(4,), kernel_initializer='random_uniform',
                bias_initializer='random_uniform')
        self.l2 = Dense(30, kernel_initializer='random_uniform',
                bias_initializer='random_uniform')
        self.l3 = Dense(num_actions, kernel_initializer='random_uniform',
                bias_initializer='random_uniform')

    def call(self, input):
        feat = tf.nn.relu(self.l1(input))
        feat = tf.nn.relu(self.l2(feat))
        value = self.l3(feat)
        return value

primary_network = QN()
target_network = QN()


def choose_action(state, primary_network, eps):
    if random.random() < eps:
        return random.randint(0, num_actions - 1)
    else:
        state = np.expand_dims(np.array(state),axis=0) #otherwise throuhg eerror..
        return np.argmax(primary_network(state))


Example choosing actions.

In [32]:
env.reset()
s, r, _, _ = env.step(env.action_space.sample())
choose_action(s, primary_network, 1)

In [ ]:
def train(primary_network, memory, tarket_network):
    if memory.num_samples < BATCH_SIZE*3:
        return 0
    batch = memory.sample(BATCH_SIZE)
    states = np.array(val[0] for val in batch)
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([(np.zeros(state_size)
                             if val[3] is None else val[3]) for val in batch])
   
    prim_qt = primary_network(np.expand_dims(states,axis=0)) # Q_t[s,a]
    prim_qtp1 = primary_network(np.expand_dims(next_states,axis=0)) #Q_{t+1}[s,a]
        
    updates = rewards
    valid_idxs = np.array(next_states).sum(axis=1) != 0
    batch_idxs = np.arange(BATCH_SIZE)
    
    opt_q_tp1_eachS = np.argmax(np.squeeze(prim_qtp1.numpy()), axis=0)
    q_from_target = target_network(np.expand_dims(next_states), axis=0)
    
    updates[valid_idx] += GAMMA*np.squeeze(q_from_target.numpy())[valid_idx, opt_q_tp1_eachS[valid_idx]]
    
    target_q = np.squeeze(prim_qt.numpy())
    target_q[batch_idxs, actions] = updates
    
    
    

Take the greedy... you have to flatten the result

In [60]:
primary_network(np.expand_dims(np.array(s),axis=0)).numpy().flatten()

array([ 0.03240588, -0.01948925], dtype=float32)

In [70]:
st=[]
rs = []
for i in range(6):

    env.reset()
    s, r, _, _ = env.step(env.action_space.sample())
    st.append(s)
    rs.append(r)
st = np.array(st)
st = np.expand_dims(st, axis=0)

In [66]:
q_values = primary_network(st).numpy()
print(q_values)
qtarget = np.squeeze(q_values)

[[[ 0.03242542 -0.01943645]
  [ 0.0324225  -0.01945483]
  [ 0.03222738 -0.01931404]
  [ 0.03220891 -0.01935516]
  [ 0.03243838 -0.01942815]
  [ 0.03221222 -0.01934218]]]


In [68]:
qtarget[np.array([1,3]),:]

array([[ 0.0324225 , -0.01945483],
       [ 0.03220891, -0.01935516]], dtype=float32)

In [69]:
np.amax(qtarget[np.array([1,3]),:],axis=1)

array([0.0324225 , 0.03220891], dtype=float32)

In [106]:
st=[]
ns=[]
rs = []
for i in range(6):
    env.reset()
    s, r, _, k = env.step(env.action_space.sample())
    st.append(s)
    rs.append(r)
    ns.append(np.random.random(4))
st = np.array(st)
ns = np.array(ns)

In [107]:
prim_qt = primary_network(np.expand_dims(st, axis=0))
prim_qtp1 = primary_network(np.expand_dims(ns, axis=0))

In [109]:
target_q = prim_qt.numpy()

In [111]:
updates = np.array(rs)

In [112]:
prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)

In [114]:
np.unravel_index(prim_action_tp1, prim_qtp1.shape)

(array([[0, 0]]), array([[1, 1]]), array([[0, 0]]))

In [115]:
prim_qtp1

<tf.Tensor: id=1017, shape=(1, 6, 2), dtype=float32, numpy=
array([[[ 0.03243664, -0.0192413 ],
        [ 0.03253958, -0.01921705],
        [ 0.0326048 , -0.01918852],
        [ 0.03226434, -0.01971841],
        [ 0.03234761, -0.01968653],
        [ 0.03209767, -0.01951555]]], dtype=float32)>

In [116]:
qt = prim_qtp1

In [119]:
qtt1 = np.squeeze(qt.numpy())

In [120]:
qtt1

array([[ 0.03243664, -0.0192413 ],
       [ 0.03253958, -0.01921705],
       [ 0.0326048 , -0.01918852],
       [ 0.03226434, -0.01971841],
       [ 0.03234761, -0.01968653],
       [ 0.03209767, -0.01951555]], dtype=float32)

In [123]:
np.argmax(qtt1, axis=1)

array([0, 0, 0, 0, 0, 0])